In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm

In [2]:
df = pd.read_csv('../data/household_power_consumption.txt', sep=';', low_memory=False)

df = df.replace('?', 0)
df = df.replace(np.nan, 0)

df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
df[[
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]] = df[[
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]].astype(float)

In [3]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.840,18.400,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.630,23.000,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.290,23.000,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.740,23.000,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.680,15.800,0.0,1.0,17.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date                   datetime64[ns]
 1   Time                   object        
 2   Global_active_power    object        
 3   Global_reactive_power  object        
 4   Voltage                object        
 5   Global_intensity       object        
 6   Sub_metering_1         float64       
 7   Sub_metering_2         float64       
 8   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 142.5+ MB


In [5]:
df_sub = df[[
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]]

df_date = df['Date']
df_time = df['Time']

In [6]:
times = sorted(list(set(df_time)))
dates = sorted(list(set(df_date)))

print(len(dates))
print(len(times))

def sort_time(x):
    return times.index(x)

df_sub['time(sort)'] = df_time.apply(sort_time)

def sort_date(x):
    return dates.index(x)

df_sub['date(sort)'] = df_date.apply(sort_date)

1442
1440


C:\Users\KentoUrano\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\KentoUrano\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [7]:
def aggregate_time(sep):
    
    XX = np.zeros([int(len(times) / sep), len(dates), 3])
    
    for i in tqdm(range(int(len(df_sub)))):
        rec = df_sub.iloc[i]
        XX[
            int(rec['time(sort)'] // sep),
            int(rec['date(sort)']),
            :
        ] += rec[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]

    return XX

In [8]:
XX_pre = aggregate_time(sep=5)

100%|███████████████████████████████████████████████████████████████████████| 2075259/2075259 [37:53<00:00, 912.60it/s]


In [9]:
XX = []
for t in range(96):
    XX.append(
        np.r_[XX_pre[3 * t], XX_pre[3 * t + 1], XX_pre[3 * t + 2]]
    )
XX = np.array(XX)
XX = np.log(1 + XX)

In [10]:
with open('../data/XX_house.pkl', 'wb') as f:
    pkl.dump(XX, f)